<a href="https://colab.research.google.com/github/larissavaladao/time_series_curuai/blob/main/5.%20parameters_by_period/7.land_cover_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importar pacotes

In [1]:
import geopandas as gpd
import pandas as pd
import math
import ee
import geemap
import json

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-curuai2')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import image collections

Landsat 7 - PY6S

In [4]:
landsat7 = ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD7/Landsat7")\
            .select([ 'B1', 'B2', 'B3', 'B4', 'B5', 'B7'])
print(ee.Date(landsat7.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat7.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2000-01-24T13:46:48
2024-01-16T11:16:49


Landsat 8 PY6S

In [5]:
landsat8 = (ee.ImageCollection("projects/ee-curuai/assets/Py6S/LD8/Landsat8")
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']))
print(ee.Date(landsat8.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat8.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2013-05-11T13:55:54
2024-12-12T13:47:48


Landsat 9 PY6S

In [6]:
landsat9 = (ee.ImageCollection("projects/ee-curuai2/assets/Py6S/LD9/Landsat9")
            .select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7']))
print(ee.Date(landsat9.sort('system:time_start',True).first().get('system:time_start')).format().getInfo())
print(ee.Date(landsat9.sort('system:time_start',False).first().get('system:time_start')).format().getInfo())

2021-11-06T13:49:55
2024-12-27T13:54:34


# Transform into remote sensing reflectance

In [7]:
def rs_reflectance (img):
  '''#dividir imagem corrigida por pi
  Rrs_sat_ac = Rsat_ac / pi
  '''
  Rrs = img.divide(math.pi).select(['blue_mean','green_mean','red_mean','nir_mean','swir1','swir2'])


  return (Rrs
          .copyProperties(img,['system:time_start','CLOUD_COVER',"system:index"]))

# Padronize band names

In [8]:
name_bands = ['blue_mean','green_mean','red_mean','nir_mean','swir1','swir2']

Landsat 7

In [12]:
#renomear bandas
ld7 = landsat7.map(lambda img: img.rename(name_bands)).map(rs_reflectance)
display(ld7.size().getInfo())

1366

Landsat 8

In [13]:
ld8 = landsat8.map(lambda img: img.rename(name_bands)).map(rs_reflectance)
display(ld8.size().getInfo())

743

landsat 9

In [14]:
ld9 = landsat9.map(lambda img: img.rename(name_bands)).map(rs_reflectance)
display(ld9.size().getInfo())

208

# Juntar coleções landsat

In [15]:
merge_col = ld8.merge(ld7).merge(ld9).sort('system:time_start')
merge_col.limit(5)

In [16]:
merge_col.size()

# Importar dados de periodo baseado na vazao de obidos

In [18]:
#import period dates
df_period_limits = pd.read_csv('/content/drive/MyDrive/CURUAI_PROCESS/water_period_limits.csv').drop(columns=['Unnamed: 0'])
# df_period_limits['Data'] = pd.to_datetime(df_period_limits['Data'])
df_period_limits

,Data,type,day,month,year,longitude,latitude
0,2000-01-13 10:43:38.181818240,LW_to_R,13,1,2000,-55.5131,-1.9192
1,2000-02-29 18:17:42.295081984,R_to_HW,29,2,2000,-55.5131,-1.9192
2,2000-08-24 20:45:42.857142784,HW_to_F,24,8,2000,-55.5131,-1.9192
3,2000-09-28 10:23:44.598930432,F_to_LW,28,9,2000,-55.5131,-1.9192
4,2001-01-05 16:27:54.418604672,LW_to_R,5,1,2001,-55.5131,-1.9192
...,...,...,...,...,...,...,...
95,2023-08-25 18:37:31.737451520,F_to_LW,25,8,2023,-55.5131,-1.9192
96,2024-01-26 18:57:28.739495680,LW_to_R,26,1,2024,-55.5131,-1.9192
97,2024-03-25 23:32:02.330097152,R_to_HW,25,3,2024,-55.5131,-1.9192
98,2024-04-06 10:42:51.428571648,HW_to_F,6,4,2024,-55.5131,-1.9192


# Fazer mosaico definido por cota

In [19]:
list_images = []
for i in range(0,len(df_period_limits)-1):
  # print(df_period_limits['Data'][i][0:10])
  # print(df_period_limits['Data'][i+1][0:10])

  image = (merge_col.filter(ee.Filter.date(df_period_limits['Data'][i][0:10],df_period_limits['Data'][i+1][0:10]))
              .median())
  if int(image.bandNames().length().getInfo()) > 0:
    list_images.append(image
              .set('year', str(df_period_limits['year'][i]))
              .set('month_init', str(df_period_limits['month'][i]))
              .set('month_end', str(df_period_limits['month'][i+1]))
              .set('system:time_start', ee.Date.parse('yyyy-MM-dd',str(df_period_limits['Data'][i][0:10])))
              .set('time_start', str(df_period_limits['Data'][i][0:10]))
              .set('time_finish', str(df_period_limits['Data'][i+1][0:10]))
              .set('band_count',image.bandNames().length()))



len(list_images)

98

In [20]:
list_images.append(merge_col.filterDate('2024-05-30','2025-01-01').median()
              .set('year', str(2024))
              .set('month_init', str('05'))
              .set('month_end', str(12))
              .set('system:time_start', ee.Date.parse('yyyy-MM-dd',str('2024-05-30')))
              .set('time_start', str('2024-05-30'))
              .set('time_finish', str('2025-01-01'))
              .set('band_count',image.bandNames().length()))

In [21]:
period_mosaics =  ee.ImageCollection.fromImages(ee.List(list_images))
period_mosaics.size()

In [22]:
Map = geemap.Map(basemap='HYBRID')

Map.addLayer(period_mosaics.first(),{},'teste')
Map


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

# Classificar Imagens

In [ ]:
# classe

Desmistura espectral? k means? random forest? indice espectral?